**Task 2:Speech Emotion Recognition**

**Import The Libaries**

In [147]:
import librosa
import soundfile as sf
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


**Extract features (mfcc, chroma, mel) from a sound  file**


In [148]:

def extract_feature(file_name, mfcc=True, chroma=True, mel=True):
    with sf.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        result = np.array([])

        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
    return result


**Emotions in the RAVDESS dataset**

In [149]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']



**Load the data and extract features for each sound file**

In [160]:

def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/speech-emotion-recognition-ravdess-data.zip (Unzipped Files)/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)


**Split the dataset into training and testing sets**

In [161]:

import numpy as np
x = np.random.rand(100, 10)  # Sample feature data (100 samples with 10 features each)
y = np.random.randint(0, 2, size=100)  # Sample label data (binary labels)

# Split the dataset into training and testing sets with a 20% test size
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=9)



In [152]:


def load_data(test_size=0.2):
    x, y = [], []
    for file_path in glob.glob("//content//drive//MyDrive//speech-emotion-recognition-ravdess-data.zip (Unzipped Files)/Actor_*//*.wav"):
        file_name = os.path.basename(file_path)
        emotion = emotions[file_name.split("-")[2]]
        x.append(file_path)
        y.append(emotion)

    x_features = [extract_feature(file) for file in x]

    if len(x_features) == 0 or len(y) == 0:
        raise ValueError("No data found. Check if the file paths are correct and the data is loaded properly.")

    x_train, x_test, y_train, y_test = train_test_split(x_features, y, test_size=0.2, random_state=42)

    return np.array(x_train), np.array(x_test), np.array(y_train), np.array(y_test)


**features extracted**

In [153]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')


Features extracted: 10


**Initialize the Multi Layer Perceptron Classifier**

In [154]:

model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)


**Train the model**

In [155]:

model.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:605: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

**Predict for the test set**

In [156]:

y_pred=model.predict(x_test)


**Calculate the accuracy of our model**

In [157]:

accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)


In [158]:
accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 70.00%
